# Wine Reviews

In [ ]:
#importo le librerie necessarie
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Importazione del file CSV
dataset = pd.read_csv("winemag-data-130k-v2.csv")
dataset = dataset.drop(columns=["Unnamed: 0"])

# Esplorazione iniziale dei dati

### Dati mancanti o inconsistenti
##### Controllo le righe con dati errati o inconsistente (ad esempio eventuali prezzi negativi, ...).

In [ ]:
dataset.info()

In [ ]:
dataset.describe(include='all')

### Rimozione delle righe che contengono NaN per prezzo, punteggio e provenienza
##### Essendo informazioni importanti nella vendita di un vino vengono rimosse le righe prive di tali informazioni

In [ ]:
dataset = dataset.dropna(subset=['price', 'points', 'country'])

In [ ]:
# Rivisualizzazione dei dati dopo il drop NaN
dataset.info()

In [ ]:
dataset.describe(include='all')

### Valutazione della distribuzione dei vini suddivisi per i diversi country in tabella
##### Do uno sguardo a quali sono i paesi piu rappresentati all'interno della tabella per avere una panoramica di quali paesi producono piu vino

In [ ]:
# Suddivido i vini per 'country'
groups_country = dataset.groupby("country").size()
# Creo un grafico a barre per visualizzare la distribuzione
fig = px.bar(groups_country.sort_values(ascending = False),title="Vini per paese", width=800, height=600)
fig.update_layout(showlegend=False)
fig.show()

Il grafico sottolinea una forte concentrazione della produzione o del valore dei vini in pochi paesi, principalmente Stati Uniti, Francia e Italia, seguiti da Spagna, Portogallo, Chile, Argentina, Austra, Australia Germania, Nuova Zelanda e Sud Africa. Questi dominano il mercato, mentre la maggior parte delle altre nazioni registra una produzione molto limitata. La distribuzione dei dati appare quindi sbilanciata, con un netto divario tra il gruppo di testa e tutti gli altri paesi.

### Valutazione della distribuzione dei prezzi e dei punteggi
##### 1. Guardo qual'è la distribuzione dei prezzi all'interno del dataset.
##### 2. Faccio lo stesso per il punteggio immaginando che esista un range entro il quale scegliere i punti da assegnare (anche osservato con il metodo  dataset.describe()).
##### 3. Infine plotto con uno scatter il prezzo dei vini (asse y) in relazione al loro punteggio (asse x) per iniziare a valutare una possibile correlazione tra le due grandezze

In [ ]:
# Creo una figura con 2 riga e 2 colonne per rappresentare i tre grafici in un unico output.
# Questo può facilitare la visualizzazione e il confronto tra i grafici
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=("Distribuzione dei Prezzi", "Distribuzione dei Punteggi", "Prezzo vs Punteggio"),
    shared_yaxes=False  # Non condividere gli assi Y
)

# Avendo osservato con il metodo .describe() una grande differenza tra min e max dei valori di prezzo e un'elevata dev. standard 
# decido di utilizzare un grafico a violino per la distribuzione dei prezzi. Questo mi permette di visualizzare in uno spazio ristretto eventuali valori
# anche molto distanti tra loro
fig_violino = px.violin(dataset, y="price", points="all", labels={"price": "Prezzo"})

# Aggiungo il grafico a violino alla figura
for trace in fig_violino.data:
    fig.add_trace(trace, row=1, col=1)

# Aggungo i titoli agli assi del grafico a violino
fig.update_xaxes(title_text="Distribuzione dei Prezzi", row=1, col=1)
fig.update_yaxes(title_text="Frequenza", row=1, col=1)

# Al contrario dei prezzi, i punteggi oscillano tra valori non molto distanti tra loro quindi decido di usare un Istogramma per la
# visualizzazione della distribuzione dei punteggi
fig_hist = px.histogram(dataset, x="points", nbins=30, histnorm='density', opacity=0.6, labels={"points": "Punteggio"})
# Aggiungo l'istogramma alla figura
for trace in fig_hist.data:
    fig.add_trace(trace, row=1, col=2)

# Aggiungo i titoli agli assi dell'istogramma
fig.update_xaxes(title_text="Punteggio", row=1, col=2)
fig.update_yaxes(title_text="Densità", row=1, col=2)

# Scatter plot: Prezzo vs Punteggio. Lo scatter permette di mettere in evidenza una possibile correlazione tra grandezze che, se presente, mostra
# una distribuzione dei punto lungo una curva di funzione.
scatter = px.scatter(dataset, x="points", y="price", title="Prezzo vs Punteggio", labels={"points": "Punteggio", "price": "Prezzo"})
# Aggiungo lo scatter plot alla figura
for trace in scatter.data:
    fig.add_trace(trace, row=2, col=1)

# Aggiungo i titoli agli assi dello scatter plot
fig.update_xaxes(title_text="Punteggio", row=2, col=1)
fig.update_yaxes(title_text="Prezzo", row=2, col=1)



# Miglioramenti alla leggibilità
fig.update_layout(
    showlegend=False,
    width=1000,  # Larghezza complessiva della figura
    height=1000, # Altezza della figura
    title="Analisi della Distribuzione dei Prezzi e Punteggi"
)

# Mostro il grafico
fig.show()

Dai grafici ottenuti osserviamo come i prezzi dei vini ricadano prevalentementi all'interno di un intervallo ben evidenziato dalla figura del violino attorno al valore medio, ma si notano degli outlier che assumono valori molto elevati.  
I punteggi si distribuiscono lungo una curva che ricorda vagamente una normale con maggior frequenza di punteggi intermedi e minor incidenza di punteggi particolarmente elevati o particolarmente bassi.  
Lo scatter plot invece suggerisce una certa tendenza delle due grandezze al variare assieme ma l'assenza di una vera e propria correlazione lineare tra le due. Si vede che all'aumentare del punteggio ad aumentere non è il prezzo del vino ma l'intervallo di prezzi che raggiungono valori massimi piu alti. Detto questo è comunque possibile osservare vini che nonostante punteggi molti alti presentano prezzi ridotti e, ancora, vini molto costosi che non si posizionano tra i rating più elevati.

### Ricerca di potenziale correlazione tra prezzi e punteggi

In [ ]:
# Visualizzo la correlazione con una heatmap per la matrice di correlazione delle variabili
plt.figure(figsize=(8, 6))
sns.heatmap(dataset[['price', 'points']].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice di Correlazione tra Prezzo e Punteggio')
plt.show()

A supporto di quanto già osservato con lo scatter plot, un valore di 0.42 suggerisce che c'è una certa tendenza delle due variabili a variare insieme, ma non in modo molto forte e non linearmente.

### Visualizzare il prezzo dei vini per paese
##### Per analizzare quali paesi producono i vini più costosi ed individuare un'eventale correlazione tra prezzo e paese, utilizzo una rappresentazione a violino. Questo approccio, rispetto all'uso della media del prezzo per ciascun paese, consente di evitare interpretazioni distorte che potrebbero derivare dalla presenza di vini molto economici. Infatti, la media potrebbe non riflettere accuratamente i prezzi più alti, in quanto i vini a basso costo potrebbero abbassarla. La rappresentazione a violino, invece, mostra meglio la distribuzione dei prezzi, evidenziando i valori estremi (come i prezzi più alti) e offrendo una visione più completa del panorama dei prezzi per ogni paese.

In [ ]:
# Creo il violin plot per visualizzare la distribuzione dei prezzi per ogni paese
fig = px.violin(dataset, x="country", y="price", 
                points="all",  
                title="Distribuzione dei Prezzi dei Vini per Paese",
                labels={"country": "Paese", "price": "Prezzo"},
                color="country",  # Colori diversi per ogni paese
                width=1000, height=600)

# Ruoto le etichette dell'asse X per migliorare la visibilità
fig.update_xaxes(tickangle=45)

# Mostro il grafico
fig.show()

Il grafico evidenzia una forte disparità nei prezzi dei vini tra i vari paesi. La Francia e gli Stati Uniti si distinguono per la presenza di vini molto costosi, seguiti da Austria, Portogallo, Italia, Australia, Spagna e Ungheria, mentre paesi come Cina, Romania e Messico mostrano una predominanza di vini a basso prezzo. Questa distribuzione riflette probabilmente fattori come la qualità del vino, il mercato di riferimento e le tradizioni vinicole di ciascun paese.

### Valutazione della distribuzione dei vini suddivisi per le diverse varietà in tabella

In [ ]:
groups_variety = dataset.groupby("variety").size()
sorted_groups_variety = groups_variety.sort_values(ascending=True)
# Creo un grafico a barre verticale
fig = px.bar(sorted_groups_variety, 
             y=sorted_groups_variety.index,  # Le varietà saranno sull'asse y
             x=sorted_groups_variety.values,  # I conteggi saranno sull'asse x
             title="Varietà disponibili", 
             width=1000, 
             height=1000)

# Disabilito la legenda
fig.update_layout(showlegend=False,
                  xaxis_title = 'Numero produttori')

# Mostro il grafico
fig.show()

# Suddivisione per fasce di rarità
##### Vedendo il grafico a barre precedente capiamo che alcuni vini sono particolarmente rari tanto da avere pochissimi produttori.
##### Come strategia per la popolazione del marketplace decido di suddividere i vini per fasce di rarità 

In [ ]:
#Calcolo le statistiche delle frequenze di varietà per stabilire una soglia di rarità
# Estraggo i valori unici delle frequenze di varietà
unique_frequencies = sorted_groups_variety.unique()

# Calcolo le statistiche sui valori unici
print(f"Media dei valori unici: {unique_frequencies.mean()}\n"
      f"Mediana dei valori unici: {np.median(unique_frequencies)}\n"
      f"Q1 (primo quartile): {np.percentile(unique_frequencies, 25)}\n"
      f"Q2 (Mediana): {np.percentile(unique_frequencies, 50)}\n"
      f"Q3 (terzo quartile): {np.percentile(unique_frequencies, 75)}\n"
      f"Deviazione standard: {np.std(unique_frequencies)}")

# Calcolo quartinli 
percentile_25 = np.percentile(unique_frequencies, 25)
percentile_50 = np.percentile(unique_frequencies, 50)
percentile_75 = np.percentile(unique_frequencies, 75)

print(f"\nPercentile 25: {percentile_25}")
print(f"Percentile 50: {percentile_50}")
print(f"Percentile 75: {percentile_75}")


In [ ]:
#Stabiliamo le 4 fasce di rarità con i quartili
varietà_rare = sorted_groups_variety[sorted_groups_variety.values < percentile_25]
varietà_poco_comuni = sorted_groups_variety[(sorted_groups_variety.values > percentile_25)  & (sorted_groups_variety.values < percentile_50)]
varietà_comuni = sorted_groups_variety[(sorted_groups_variety.values > percentile_50)  & (sorted_groups_variety.values < percentile_75)]
varietà_popolari = sorted_groups_variety[sorted_groups_variety.values > percentile_75]

Ho valutato la distribuzione delle frequenze delle diverse varietà di vini e le ho divise, utilizzando i quartili, in quattro classi di popolarità:
1. Vini rari: varietà di vini prodotti da meno di 39 produttori in tutto il mondo
2. Vini poco comuni: varietà di vini prodotti da un numero di produttori compreso tra 40 e 121
3. Vini comuni: varietà di vini prodotti da un numero di produttori compreso tra 122 e 541
4. Vini popolari: varietà di vini prodotti da più di 541 produttori nel mondo

# Vini rari

In [ ]:
# Conto quante varietà di vini rari esistono
varietà_rare.count()

In [ ]:
# Li estraggo dal dataset completo
vini_rari = dataset[dataset['variety'].isin(varietà_rare.index)]
#vini_rari.sort_values('points', ascending = False).head(10)

Trattandosi di vini rari, che quindi verranno venduti ad alto prezzo e a una fetta ristretta della clientela ho deciso di selezionarne solo 5 per il mio marketplace. Volendo selezionare solo 5 vini rari si valutano quelli con i 5 punteggi più alti e per ciascun punteggio si predilige il vino più economico in modo da poter avere un profitto piu elevato al momento della vendita.

In [ ]:
# Seleziono i 5 score migliori
migliori_scores_vini_rari = np.sort(vini_rari.points.unique())[-5:]

# Filtro i vini con i migliori punteggi
migliori_vini_rari = vini_rari[vini_rari.points.isin(migliori_scores_vini_rari)]

# Raggruppo per 'points', 'country' e 'price' e conto il numero di vini per ciascuna combinazione
conteggio_vini_per_punteggio_country = migliori_vini_rari.groupby(['points', 'country', 'price']).size().reset_index(name='conteggio')

# Ordino per 'points' e 'price' (all'interno di ogni punteggio)
conteggio_vini_per_punteggio_country_sorted = conteggio_vini_per_punteggio_country.sort_values(['points', 'price'], ascending=[False, False])

# Creo il grafico a barre
fig = px.bar(conteggio_vini_per_punteggio_country_sorted, 
             x='points', 
             y='conteggio', 
             color='country',  # Dividi per paese (colorazione per paese)
             hover_data={'price': True},  # Mostra il prezzo nel tooltip
             title='Conteggio Vini Rari per Punteggio e Paese', 
             labels={'points': 'Punteggio', 'conteggio': 'Conteggio Vini', 'country': 'Paese'},
             width=800, height=600)

# Mostro il grafico
fig.show()


##### Volendo rivendere i vini conviene selezionare i vini meno costosi in modo tale da avere un maggior margine di profitto

In [ ]:
min_price_per_points = migliori_vini_rari.groupby('points')['price'].min()
min_price_per_points

In [ ]:
min_price_per_points = migliori_vini_rari.groupby('points')['price'].min()
vini_rari_marketplace = pd.DataFrame()

for index, value in min_price_per_points.items():
    # Filtro i dati per i punti e il prezzo minimo
    filtered_data = vini_rari[(vini_rari['points'] == index) & (vini_rari['price'] == value)]
    
    # Concateno i dati filtrati al DataFrame esistente
    vini_rari_marketplace = pd.concat([vini_rari_marketplace, filtered_data], axis=0, ignore_index=True)
#vini_rari_marketplace

### Vini Poco Comuni

In [ ]:
# Conto quante varietà di vini poco comuni esistono
varietà_poco_comuni.count()

In [ ]:
# Li estraggo dal dataset completo
vini_poco_comuni = dataset[dataset['variety'].isin(varietà_poco_comuni.index)]
#vini_poco_comuni.sort_values('points', ascending = False).head(10)

In questo caso si valutano i vini poco comuni, di questi voglio sempre i top di gamma e dunque seleziono i 5 score più elevati.
Volendo però avere nel marketplace un numero di questi vini più elevato rispetto a quello dei vini rari ho deciso che per ogni punteggio prenderò un vino per ciascun paese e in caso di più vini dello stesso paese quello con il prezzo più basso, sempre per poter ottenere un miglior profitto. 

In [ ]:
#Guardo i migliori score
migliori_scores_vini_poco_comuni = np.sort(vini_poco_comuni.points.unique())[-5:]

# Filtro i vini con i migliori punteggi
migliori_vini_poco_comuni = vini_poco_comuni[vini_poco_comuni.points.isin(migliori_scores_vini_poco_comuni)]

# Raggruppo per 'points', 'country' e 'price' e conta il numero di vini per ciascuna combinazione
conteggio_vini_per_punteggio_country = migliori_vini_poco_comuni.groupby(['points', 'country', 'price']).size().reset_index(name='conteggio')

# Ordino per 'points' e 'price' (all'interno di ogni punteggio)
conteggio_vini_per_punteggio_country_sorted = conteggio_vini_per_punteggio_country.sort_values(['points', 'price'], ascending=[False, False])

# Creo il grafico a barre
fig = px.bar(conteggio_vini_per_punteggio_country_sorted, 
             x='points', 
             y='conteggio', 
             color='country',  # Dividi per paese (colorazione per paese)
             hover_data={'price': True},  # Mostra il prezzo nel tooltip
             title='Conteggio Vini Rari per Punteggio e Paese Ordinato per Prezzo', 
             labels={'points': 'Punteggio', 'conteggio': 'Conteggio Vini', 'country': 'Paese'},
             width=800, height=600)

# Mostro il grafico
fig.show()


In [ ]:
vini_poco_comuni_marketplace = pd.DataFrame()
# Itero attraverso ogni gruppo di punti
# Voglio per ogni punteggio prendere il vino che costa meno per ogni paese. 
# Itero su ogni punteggio unico
for p in conteggio_vini_per_punteggio_country.points.unique():
    # Filtro il livello con il punteggio 'p'
    p_level = conteggio_vini_per_punteggio_country[conteggio_vini_per_punteggio_country.points == p]
    
    # Raggruppo per paese e trova il prezzo minimo per ogni paese
    group_country = p_level.groupby('country').min()
    for index, row in group_country.iterrows():
        country = index  # L'indice è il paese
        price = row.iloc[1]  # Accedo al prezzo (seconda colonna)
        
        filtered_data = migliori_vini_poco_comuni[
            (migliori_vini_poco_comuni['country'] == country) & 
            (migliori_vini_poco_comuni['price'] == price) & 
            (migliori_vini_poco_comuni['points'] == p)
        ][:1] #A parità di punteggio, prezzo e paese ne prendo solo uno.
        
        # Aggiungo le righe selezionate al DataFrame finale
        vini_poco_comuni_marketplace = pd.concat([vini_poco_comuni_marketplace, filtered_data], axis=0, ignore_index=True)
# Visualizzo il DataFrame finale con i vini selezionati
vini_poco_comuni_marketplace 

### Vini Comuni

In [ ]:
# Conto quante varietà di vini comuni esistono
varietà_comuni.count()

In [ ]:
# Li estraggo dal dataset completo
vini_comuni = dataset[dataset['variety'].isin(varietà_comuni.index)]
#vini_comuni.sort_values('points', ascending = False).head(10)

Tra i vini comuni cerco di averne 3 per varietà con il miglior rapporto qualità/prezzo

In [ ]:
# Raggruppa per varietà e mostra i punteggi disponibili
 #Creo un box plot che mostra la distribuzione dei punteggi per varietà
fig = px.box(vini_comuni, x="variety", y="points",
             title="Distribuzione dei Punteggi per Varietà di Vino",
             labels={"variety": "Varietà di Vino", "points": "Punteggio"},
             width=1000, height=600)

fig.update_layout(xaxis_tickangle=45)

# Visualizzo il grafico
fig.show()


In [ ]:
# Aggiungo una nuova colonna con il rapporto tra punti e prezzo
vini_comuni['points_to_price'] = vini_comuni['points'] / vini_comuni['price']

# Seleziono i 3 vini con il miglior rapporto per ciascuna varietà
vini_comuni_marketplace = vini_comuni.sort_values('points_to_price', ascending=False) \
                                      .groupby('variety').head(3)

# Visualizzo i vini selezionati
vini_comuni_marketplace


### Vini Popolari

In [ ]:
# Conto quante varietà di vini comuni esistono
varietà_popolari.count()

In [ ]:
# Li estraggo dal dataset completo
vini_popolari = dataset[dataset['variety'].isin(varietà_popolari.index)]
#vini_popolari.sort_values('points', ascending = False).head(10)

Tra i vini popolari cerco di averne 5 per varietà con il miglior rapporto qualità/prezzo

In [ ]:
# Raggruppo per varietà e mostra i punteggi disponibili
 #Creo un box plot che mostra la distribuzione dei punteggi per varietà
fig = px.box(vini_popolari, x="variety", y="points",
             title="Distribuzione dei Punteggi per Varietà di Vino",
             labels={"variety": "Varietà di Vino", "points": "Punteggio"},
             width=1000, height=600)

fig.update_layout(xaxis_tickangle=45)

# Visualizzo
fig.show()


In [ ]:
# Aggiungo una nuova colonna con il rapporto tra punti e prezzo
vini_popolari.loc[:, 'points_to_price'] = vini_popolari['points'] / vini_popolari['price']

# Seleziono i 3 vini con il miglior rapporto per ciascuna varietà
vini_popolari_marketplace = vini_popolari.sort_values('points_to_price', ascending=False) \
                                         .groupby('variety').head(5)

# Visualizz i vini selezionati
vini_popolari_marketplace


### Creo il marketplace
##### Concateno i quattro gruppi di vini appartenenti alle 4 fasce di popolarità selezionate

In [ ]:
my_marketplace = pd.concat([vini_rari_marketplace, vini_poco_comuni_marketplace, vini_comuni_marketplace, vini_popolari_marketplace], ignore_index=True, axis = 0)

In [ ]:
my_marketplace.info()

In [ ]:
my_marketplace.describe(include='all')

### Mi assicuro di avere una vasta gamma di vini alla portata di tutti

In [ ]:
len(my_marketplace[my_marketplace.price < 10])

### Do uno sguardo alle distribuzioni di prezzo, punteggio e varietà nel mio marketplace

In [ ]:
# Creo una figura con 2 riga e 2 colonne per rappresentare i tre grafici in un unico output.
# Questo può facilitare la visualizzazione e il confronto tra i grafici
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=("Distribuzione dei Prezzi", "Distribuzione dei Punteggi", "Distribuzione varietà", "Provenienza dei vini"),
    shared_yaxes=False  # Non condividere gli assi Y
)

#Violino
fig_violino = px.violin(my_marketplace, y="price", points="all", labels={"price": "Prezzo"})
# Aggiungo il grafico a violino alla figura
for trace in fig_violino.data:
    fig.add_trace(trace, row=1, col=1)
# Aggungo i titoli agli assi del grafico a violino
fig.update_xaxes(title_text="Distribuzione dei Prezzi", row=1, col=1)
fig.update_yaxes(title_text="Frequenza", row=1, col=1)

#Istogramma
fig_hist = px.histogram(my_marketplace, x="points", nbins=30, histnorm='density', opacity=0.6, labels={"points": "Punteggio"})
# Aggiungo l'istogramma alla figura
for trace in fig_hist.data:
    fig.add_trace(trace, row=1, col=2)
# Aggiungo i titoli agli assi dell'istogramma
fig.update_xaxes(title_text="Punteggio", row=1, col=2)
fig.update_yaxes(title_text="Densità", row=1, col=2)

#Barplot 
groups_variety = my_marketplace.groupby("variety").size()
sorted_groups_variety = groups_variety.sort_values(ascending=True)
# Creo un grafico a barre verticale
fig_bar = px.bar(sorted_groups_variety, 
             y=sorted_groups_variety.index,  
             x=sorted_groups_variety.values,  
             title="Varietà disponibili")

# Disabilito la legenda
fig_bar.update_layout(showlegend=False,
                  xaxis_title = 'Numero produttori')

# Aggiungo il bar plot alla figura
for trace in fig_bar.data:
    fig.add_trace(trace, row=2, col=1)

# Aggiungo i titoli agli assi del bar plot
fig.update_xaxes(title_text="Frequenza", row=2, col=1)
fig.update_yaxes(title_text="Varietà", row=2, col=1)

# Distribuzione per paese
# Suddivido i vini per 'country'
groups_country = my_marketplace.groupby("country").size()
# Creo un grafico a barre per visualizzare la distribuzione
fig_paese = px.bar(groups_country.sort_values(ascending = False),title="Vini per paese")
fig_paese.update_layout(showlegend=False)

# Aggiungo il plot alla figura
for trace in fig_paese.data:
    fig.add_trace(trace, row=2, col=2)


# Miglioramenti alla leggibilità
fig.update_layout(
    showlegend=False,
    width=1000,  # Larghezza complessiva della figura
    height=1200, # Altezza della figura
    title="Analisi della Distribuzione dei Prezzi e Punteggi"
)

# Mostro il grafico
fig.show()

L’analisi complessiva suggerisce che questo negozio offre una selezione di vini ampia e bilanciata. I prezzi coprono un’ampia gamma, con un focus su bottiglie accessibili, ma con alcune proposte di lusso per i clienti più esigenti. La qualità è mediamente alta, come indicato dai punteggi, e la varietà soddisfa sia chi cerca etichette tradizionali sia chi desidera esplorare vini meno conosciuti. Nel complesso, si tratta di un’offerta curata e variegata, capace di attrarre un pubblico diversificato, dagli appassionati ai grandi intenditori.